In [6]:
class twitterStuff:
    import tweepy
    import json
    import time
    def pullTweets(screenName='dadsaysjokes', tweetCutoff=3200):
        ''''
        A function to pull tweets objects based on a user's screen name and number of tweets. 

            INPUTS:
            - screenName = the user's screen name(handle) to pull (string)
            - tweetCutoff = this is an integer which represents how many tweets will be pulled (default = 3200 tweets)

            OUTPUTS:
            - screenName_tweets.json (where 'screenName' is the user handle input)

        ''' ''
        # Authorize with application tokens
        auth = tweepy.OAuthHandler(
            'nLQR3kn2wZaGnIx96caKCO3v3',
            'xkzbUfh43hhjy4QhGlDO1ZgABUUrxZ17fFAnKEvl3n8hNTXjxs')
        auth.set_access_token(
            '1127969462521618434-6fRp67ll6LZWaFa8bi0zXGpDORHJ5X',
            'M1OOzWaegn5Cs5o5wjtsYn2m8LdRL0E08ahlocQLCWjbt')

        # initialize tweepy into 'api'
        api = tweepy.API(auth)

        # iterate through a user's timeline and grab tweets, placing them into all_tweets
        all_tweets = []

        try:
            for tweet in tweepy.Cursor(
                    api.user_timeline,
                    screen_name=screenName,
                    include_entities=True,
                    tweet_mode='extended',
                    wait_on_rate_limit=True,
                    wait_on_rate_limit_notify=True).items(tweetCutoff):
                # append each tweet to all_tweets
                all_tweets.append(tweet)

            # write the json
            with open('{}_tweets.json'.format(screenName), 'w', encoding='utf-8') as f:
                for tweet in all_tweets:
                    f.write(json.dumps(tweet._json) + '\n')
                print('{}_tweets.json was successfully created.'.format(
                    screenName))

        except tweepy.TweepError as error:
            reason = error.reason
            response = error.response
            apiCode = error.api_code
            errorResponse = '''
            ********
            The following TweepError has been encountered for handle: {}
            Reason: {}
            Response: {}
            API Code: {}
            See https://developer.twitter.com/en/docs/basics/response-codes.html for more info.
            --->Moving on...
            ********
            '''.format(screenName, reason, response, apiCode)
            print(errorResponse)
        except Exception as e:
            print(e)
            pass

In [ ]:
# Change the display parameters so that you can properly inspect output below
display.max_colwidth = 200
display.max_columns = 'None'
display.max_info_columns = 200
pd.set_option('display.max_rows', 700)

In [ ]:
import os
import pandas as pd

os.chdir('C:\\Users\\mdeve\\OneDrive\\Documents\\Twitter_Tweepy\\data\\Network_Ideologies_Data\\congressData')

handlesFile = 'C:\\Users\\mdeve\\OneDrive\\Documents\\Twitter_Tweepy\\data\\Network_Ideologies_Data\\twitterAllHandles_7.20.19.csv'

data = pd.read_csv(handlesFile)

for handle in data.Twitter:
    try:
        twitterStuff.pullTweets(handle)
    except Exception as e:
        print(e)
        continue


# Check which handles did not create files

In [ ]:
# get a list of the files in the directory
files = os.listdir()

# put that list into a dataframe
handleFiles = pd.DataFrame(files, columns={'Twitter'})

# split on the '_tweets to get just the handles - split into different columns
handless = handleFiles['Twitter'].str.split('_tweets', expand=True)

# take only the handles column, lowercase them all
files_files = handless[0].str.lower()

# lowercase all handles in the original list of handles as well
data['Twitter'] = data['Twitter'].str.lower()

# turn the list of handles from files into a dataframe to allow for use of 'isin()' below
files_files = pd.DataFrame(files_files)

In [ ]:

missingHandles = data['Twitter'][data['Twitter'].isin(files_files[0]) == False]
missingHandles

In [ ]:

len(missingHandles)

In [ ]:
# Rerun the code again, as a double check, on the remaining handles
# in 'missingHandles' to ensure all handles are captured. 

In [ ]:
import os
import pandas as pd

os.chdir('C:\\Users\\mdeve\\OneDrive\\Documents\\Twitter_Tweepy\\data\\Network_Ideologies_Data\\congressData')

for handle in missingHandles:
    try:
        twitterStuff.pullTweets(handle)
    except Exception as e:
        print(e)

In [ ]:

# Save these missing handles to a dataframe
missingHandles = pd.DataFrame(missingHandles)
missingHandles.to_csv('missingHandles_2.csv')